# Test Tranfers

In [2]:
import numpy as np 
import json
import os 
import sys
import random
import math

In [3]:
from utils_pose_alignment import get_urdf_path, convert_gripper_to_aligned_pose, convert_aligned_to_gripper_pose

In [3]:
test_dir = r'/home/felipe/Documents/tests'
json_directory = (r'/home/felipe/Documents/filtered_grasps')

json_files = [pos_json for pos_json in os.listdir(json_directory) if pos_json.endswith('.json')]

th = 3 
objects=set([])

grippers= {"fetch_gripper": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "franka_panda": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "sawyer": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "wsg_50": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "Barrett": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "robotiq_3finger": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "jaco_robot": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "Allegro": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "shadow_hand": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "HumanHand": {'object_id':[], 'pose':[], 'og_gripper':[]},
            "h5_hand": {'object_id':[], 'pose':[], 'og_gripper':[]}}
#print(json_files)

In [4]:
i=0
for file in json_files:
    json_path = os.path.join(json_directory,file)

    fd = open(json_path)
    dict = json.load(fd)
    #print(dict['pose'])
    objects.add(dict['object_id'])

    tmp = np.array(dict['fall_time'])
    pose = np.array(dict['pose'])
    
    pose = pose[tmp>=th]
    
    x = len(pose)
    y = min(x,5)
    samples = random.sample(range(x),y)
    pose = pose[samples]
    #print(len(pose))
    for gripper in grippers:
        if gripper == dict['gripper']:
            continue
            
        new_pose = pose 

        grippers[gripper]['object_id']+= ([dict['object_id']]* len(pose))
        grippers[gripper]['og_gripper']+= ([dict['gripper']]*len(pose))
        grippers[gripper]['pose']+= (pose.tolist())
    i+=1

In [6]:
for gripper in grippers:
    if(len(grippers[gripper]['object_id'])==0):
        continue
    gripper_dir = os.path.join(test_dir,gripper)

    object_set = {"Animal_Planet_Foam_2Headed_Dragon", "Racoon", "Nestle_Nips_Hard_Candy_Peanut_Butter"}

    j = 0
    for obj in object_set:
        output_path = os.path.join(test_dir,gripper+ '-' + obj + '.json')
        if os.path.exists(output_path): 
            continue
        
        new_json = {}
        obj_arr = np.array(grippers[gripper]['object_id'])
        obj_arr = obj_arr == obj
        
        pose_arr = np.array(grippers[gripper]['pose'])
        tmp_pose = pose_arr[obj_arr]
        tmp_pose[:,[6,3,4,5]]= tmp_pose[:,[3,4,5,6]]
        
        og_arr = np.array(grippers[gripper]['og_gripper'])
        tmp_og = og_arr[obj_arr]
        
        transformed_pose = np.zeros_like(tmp_pose)

        c =0
        for pose in tmp_pose:
            transformed_pose[c] = convert_gripper_to_aligned_pose(pose, tmp_og[c])
            transformed_pose[c] = convert_aligned_to_gripper_pose(transformed_pose[c], gripper)
            c+=1
        
        transformed_pose[:,[3,4,5,6]]= transformed_pose[:,[6,3,4,5]]
        print(transformed_pose.shape)
        new_json['gripper'] = gripper
        new_json['object_id'] = obj
        new_json['pose'] = transformed_pose.tolist()
        new_json['og_gripper'] = tmp_og.tolist()   

        
        with open(output_path,'w') as outfile:
            print("Created file: ", output_path)
            json.dump(new_json,outfile)
        
        j+=1
        

(45, 7)
Created file:  /home/felipe/Documents/tests/fetch_gripper-Nestle_Nips_Hard_Candy_Peanut_Butter.json
(50, 7)
Created file:  /home/felipe/Documents/tests/fetch_gripper-Animal_Planet_Foam_2Headed_Dragon.json
(47, 7)
Created file:  /home/felipe/Documents/tests/fetch_gripper-Racoon.json
(45, 7)
Created file:  /home/felipe/Documents/tests/franka_panda-Nestle_Nips_Hard_Candy_Peanut_Butter.json
(50, 7)
Created file:  /home/felipe/Documents/tests/franka_panda-Animal_Planet_Foam_2Headed_Dragon.json
(47, 7)
Created file:  /home/felipe/Documents/tests/franka_panda-Racoon.json
(50, 7)
Created file:  /home/felipe/Documents/tests/sawyer-Nestle_Nips_Hard_Candy_Peanut_Butter.json
(50, 7)
Created file:  /home/felipe/Documents/tests/sawyer-Animal_Planet_Foam_2Headed_Dragon.json
(50, 7)
Created file:  /home/felipe/Documents/tests/sawyer-Racoon.json
(45, 7)
Created file:  /home/felipe/Documents/tests/wsg_50-Nestle_Nips_Hard_Candy_Peanut_Butter.json
(50, 7)
Created file:  /home/felipe/Documents/test